In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import lightning.pytorch as pl
import torch.nn.functional as F
import torch.nn as nn
from typing import Any
from lightning.pytorch.utilities.types import STEP_OUTPUT, OptimizerLRScheduler
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from torch.optim.lr_scheduler import ReduceLROnPlateau

import pandas as pd
import pathlib
import numpy as np
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix

DATA_PATH = pathlib.Path("../Data")

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [24]:
df = pd.read_feather(DATA_PATH / "Classification.feather")

In [3]:
X = df.drop(columns=["timestamp", "Target"]) # this is purely a classification no time steps are needed
y = df["Target"]

# nm_undersampler = NearMiss(version=3, n_neighbors_ver3=3, n_jobs=-1) # Warning takes very long to run
# X, y = nm_undersampler.fit_resample(X, y)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
X_train_t = torch.from_numpy(X_train)
y_train_t = torch.from_numpy(y_train.to_numpy())
X_test_t = torch.from_numpy(X_test)
y_test_t = torch.from_numpy(y_test.to_numpy())

# Create the dataset object
train_ds = TensorDataset(X_train_t, y_train_t)
test_ds = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)

X_train_t.shape

In [3]:
class NN_Classifier(pl.LightningModule):
    def __init__(self, n_inputs) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(16, 256),
            nn.Tanh(),
            nn.Linear(256, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 3)
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)
    
    def training_step(self, batch, batch_idx) -> STEP_OUTPUT:
        X, y = batch
        y_pred = self(X)
        loss = F.cross_entropy(y_pred, y)
        return loss
    
    def validation_step(self, batch, batch_idx) -> STEP_OUTPUT:
        X, y = batch
        y_pred = self(X)
        y_lab = torch.argmax(F.softmax(y_pred, dim=1), dim=1)
        
        correct = (y_lab == y).sum()
        acc = correct / X.shape[0]
        
        loss = F.cross_entropy(y_pred, y)
        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True)
        return loss
    
    def configure_optimizers(self) -> OptimizerLRScheduler:
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        
        scheduler = {
            'scheduler': ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True),
            'monitor': 'val_loss',
        }
        return [optimizer], [scheduler]
        
        return optimizer

In [16]:
model = NN_Classifier(X_train_t.shape[1])
es_callback = EarlyStopping(monitor="val_loss", min_delta=0, patience=10, strict=True)
torch.set_float32_matmul_precision('medium')

trainer = pl.Trainer(accelerator="gpu", callbacks=[es_callback], max_epochs=100)
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 25.2 K
-------------------------------------
25.2 K    Trainable params
0         Non-trainable params
25.2 K    Total params
0.101     Total estimated model params size (MB)


c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 99: 100%|██████████| 1187/1187 [00:07<00:00, 155.73it/s, v_num=27, val_loss=0.517, val_acc=0.761]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1187/1187 [00:07<00:00, 155.54it/s, v_num=27, val_loss=0.517, val_acc=0.761]


In [41]:
with torch.no_grad():
    y_pred = model(X_test_t)
    y_hat = torch.argmax(F.softmax(y_pred, dim=1), dim=1).detach().cpu().numpy()
    y_true = y_test_t.numpy()

    print(classification_report(y_true, y_hat))
    print(confusion_matrix(y_true, y_hat, normalize="true"))

              precision    recall  f1-score   support

           0       0.78      0.76      0.77      6529
           1       0.75      0.87      0.80      6444
           2       0.74      0.57      0.64      3303

    accuracy                           0.76     16276
   macro avg       0.76      0.73      0.74     16276
weighted avg       0.76      0.76      0.76     16276

[[0.75509266 0.1828764  0.06203094]
 [0.09233395 0.86638734 0.04127871]
 [0.23705722 0.19709355 0.56584923]]


In [22]:
MODEL_PATH = pathlib.Path("../Model")
torch.save(model.state_dict(), MODEL_PATH / "wights_raw.pickle")

In [35]:
torch.jit.save(model.to_torchscript(method='trace', 
                                    example_inputs=X_train_t[0].view(1, -1)), 
               f = MODEL_PATH / "script_model.pt")

In [25]:
X = df.drop(columns=["timestamp", "Target"]) # this is purely a classification no time steps are needed
y = df["Target"]

scaler = StandardScaler()
X_s = scaler.fit_transform(X)

X_s

array([[-3.7855956e-01,  1.1177493e+00,  6.5752178e-01, ...,
        -5.2769203e-04, -3.8615271e-02, -4.2659864e-01],
       [-3.7855956e-01,  1.1177493e+00,  6.5752178e-01, ...,
        -5.2769203e-04, -3.8615271e-02, -4.2659864e-01],
       [-3.7790960e-01,  1.1177493e+00,  6.5752178e-01, ...,
        -5.2769203e-04, -3.8615271e-02, -4.2659864e-01],
       ...,
       [-3.7790960e-01, -7.6653486e-01,  1.4803013e-01, ...,
        -5.2769203e-04, -3.8615271e-02, -4.2659864e-01],
       [-3.7790960e-01, -7.6653486e-01,  1.4803013e-01, ...,
        -5.2769203e-04, -3.8615271e-02, -4.2659864e-01],
       [-3.7725961e-01, -7.6653486e-01,  1.4803013e-01, ...,
        -5.2769203e-04, -3.8615271e-02, -4.2659864e-01]], dtype=float32)

In [38]:
import joblib
joblib.dump(scaler, MODEL_PATH / "scaler.pkl")

['..\\Model\\scaler.pkl']

In [6]:
import joblib

MODEL_PATH = pathlib.Path("../Model")
m2 = NN_Classifier(n_inputs=16)
m2.load_state_dict(torch.load(MODEL_PATH / "wights_raw.pickle"))

scaler = joblib.load(MODEL_PATH / 'scaler.pkl')

df = pd.read_feather(DATA_PATH / "Classification.feather")

F = df.drop(columns=["timestamp", "Target"]) # this is purely a classification no time steps are needed
t = df["Target"]

X_new = F.iloc[0].to_numpy(dtype=float)
s = scaler.transform(X_new.reshape(-1, 16))
s = torch.tensor(s, dtype=torch.float32)

m2(s)


c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


tensor([[ 4.2432, -0.3997, -4.1261]], grad_fn=<AddmmBackward0>)